# Import Modules

In [28]:
import requests
from bs4 import BeautifulSoup as bs
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import pandas as pd

# Amazon Video Game Data Set (URL)

In [29]:
video_game_data = "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz"

video_game_data_request = requests.get(video_game_data)

video_game_data_request

# Load into Pandas DataFrame 

In [32]:
video_game_data = pd.read_json(
    "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz",
    compression = 'gzip',
    lines=True
)

<Response [200]>

# Set up NLP [IN PROGRESS]

In [45]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

[nltk_data] Downloading package stopwords to /Users/raja_/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/raja_/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Apply NLP Methods [BROKEN]

In [48]:
X_train = video_game_data[['reviewText']]

vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(X_train['reviewText'])
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
X_train

ValueError: np.nan is an invalid document, expected byte or unicode string.